In [1]:
import json
from datetime import datetime
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
import json
import sqlite3
from datetime import datetime

# read the json file
def load_json_lines(filename):
    records = []
    with open(filename, 'r') as file:
        for line in file:
            # Assuming each line in the file is a separate JSON object
            if line.strip():  # Skipping empty lines
                records.append(json.loads(line))
    return records

from datetime import datetime

def parse_date(unix_timestamp):
    if unix_timestamp is not None:  # Check if the timestamp is not None
        # Convert milliseconds to seconds and then to a datetime object
        return datetime.utcfromtimestamp(unix_timestamp / 1000.0).strftime('%Y-%m-%d')
    return None


def insert_data(db_path, table_name, columns, data_records):
    placeholders = ', '.join(['?'] * len(columns)) 
    column_headers = ', '.join(columns)
    insert_query = f'INSERT OR IGNORE INTO {table_name} ({column_headers}) VALUES ({placeholders});'
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()
        cur.executemany(insert_query, data_records)
        conn.commit()

db_path = '/Users/smitmalik/Documents/fetch.db'

In [3]:
# Load and transform Users data
users_json = load_json_lines('/Users/smitmalik/Downloads/users.json')
users = [
    (
        user['_id'].get('$oid', ''),  # Extracting the string representation of the ID
        1 if user.get('active', False) else 0,  # Convert boolean to integer
        parse_date(user.get('createdDate', {}).get('$date')) if user.get('createdDate', {}).get('$date') else None,  # Assuming parse_date returns a string
        parse_date(user.get('lastLogin', {}).get('$date')) if user.get('lastLogin', {}).get('$date') else None,  # Assuming parse_date returns a string
        user.get('role', ''),
        user.get('signUpSource', ''),
        user.get('state', '')
    ) 
    for user in users_json
]

In [4]:
# Load and transform Brands data
brands_json = load_json_lines('/Users/smitmalik/Downloads/brands.json')
brands_records = [
    (
        brand.get('_id', {}).get('$oid', ''),  # Accessing nested '$oid' from '_id'
        brand.get('barcode', ''),  # Directly accessing 'barcode'
        brand.get('brandCode', ''),  # Directly accessing 'brandCode'
        brand.get('category', ''),  # Directly accessing 'category'
        brand.get('categoryCode', ''),  # Directly accessing 'categoryCode'
        brand.get('cpg', {}).get('$id', {}).get('$oid', ''),  # Accessing nested '$oid' from '$id' which is nested inside 'cpg'
        brand.get('name', ''),  # Directly accessing 'name'
        brand.get('topBrand', False)  # Directly accessing 'topBrand' with default False
    ) 
    for brand in brands_json
]

In [5]:
# Load and transform Receipts data
receipts_json = load_json_lines('/Users/smitmalik/Downloads/receipts.json')
receipts_records = []
items_records = []

for receipt in receipts_json:
    receipt_id = receipt.get('_id', {}).get('$oid', '')
    receipts_records.append((
        receipt_id,
        receipt.get('bonusPointsEarned', 0),
        receipt.get('bonusPointsEarnedReason', ''),
        parse_date(receipt.get('createDate', {}).get('$date')),
        parse_date(receipt.get('dateScanned', {}).get('$date')),
        parse_date(receipt.get('finishedDate', {}).get('$date')),
        parse_date(receipt.get('modifyDate', {}).get('$date')),
        parse_date(receipt.get('pointsAwardedDate', {}).get('$date')),
        float(receipt.get('pointsEarned', 0)),
        parse_date(receipt.get('purchaseDate', {}).get('$date')),
        receipt.get('purchasedItemCount', 0),
        receipt.get('rewardsReceiptStatus', ''),
        float(receipt.get('totalSpent', 0)),
        receipt.get('userId', '')
    ))

    # Add item records
    for item in receipt.get('rewardsReceiptItemList', []):
        items_records.append((
            receipt_id,
            item.get('barcode', ''),
            item.get('description', ''),
            float(item.get('finalPrice', 0)),
            float(item.get('itemPrice', 0)),
            item.get('needsFetchReview', False),
            item.get('partnerItemId', ''),
            item.get('preventTargetGapPoints', False),
            item.get('quantityPurchased', 0),
            item.get('userFlaggedBarcode', ''),
            item.get('userFlaggedNewItem', False),
            float(item.get('userFlaggedPrice', 0)),
            item.get('userFlaggedQuantity', 0),
            item.get('userFlaggedDescription', 0)
        ))

### Insert data into the SQLite database

In [6]:
insert_data(db_path, 'user', ['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource', 'state'], users)

In [7]:
insert_data(db_path, 'brands', ['_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'cpg_id'
                               , 'name', 'topBrand'], brands_records)

In [8]:
insert_data(db_path, 'receipts', ['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 
                                  'createDate', 'dateScanned', 'finishedDate', 
                                  'modifyDate', 'pointsAwardedDate', 'pointsEarned',
                                  'purchaseDate', 'purchasedItemCount',
                                  'rewardsReceiptStatus', 'totalSpent', 'userId'], receipts_records)

In [9]:
insert_data(db_path, 'receipt_items', 
            ['receipt_id', 'barcode', 'description', 'finalPrice', 'itemPrice', 
             'needsFetchRevie', 'partnerItemId', 'preventTargetGapPoints', 
             'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem', 
             'userFlaggedPrice', 'userFlaggedQuantity', 'userFlaggedDescription'], 
            items_records)

In [10]:
import sqlite3
conn = sqlite3.connect('/Users/smitmalik/Documents/fetch.db') 

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [11]:
cursor = conn.cursor()
query = """
SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
FROM receipts
WHERE (rewardsReceiptStatus = 'FINISHED') | (rewardsReceiptStatus = 'REJECTED')
GROUP BY rewardsReceiptStatus;
"""
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)

('FINISHED', 80.85430501930502)
('REJECTED', 23.326056338028184)


### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [12]:
cursor = conn.cursor()
query = """
SELECT rewardsReceiptStatus, SUM(purchasedItemCount) AS total_items
FROM receipts
WHERE (rewardsReceiptStatus = 'FINISHED') | (rewardsReceiptStatus = 'REJECTED')
GROUP BY rewardsReceiptStatus;

"""
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)

('FINISHED', 8184)
('REJECTED', 173)


## The following questions have been answered, assuming that a direct link has been set a direct link between 'brands' and 'receipt_items' via a shared identifier like 'brandCode' or 'barcode'

### What are the top 5 brands by receipts scanned for most recent month? 

In [13]:
# cursor = conn.cursor()
# query = """
# SELECT b._id, b.name, COUNT(DISTINCT r._id) AS receipt_count
# FROM brands b
# JOIN receipt_items ri ON b.brandCode = ri.brandCode  -- using brandCode as the linking attribute
# JOIN receipts r ON ri.receipt_id = r._id
# WHERE r.dateScanned >= date('now', 'start of month', '-1 month') 
#       AND r.dateScanned < date('now', 'start of month')
# GROUP BY b._id
# ORDER BY receipt_count DESC
# LIMIT 5;
# """
# cursor.execute(query)
# rows = cursor.fetchall()
# for row in rows:
#     print(row)

### How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [14]:
# cursor = conn.cursor()
# query = """
# WITH RecentMonth AS (
#     SELECT b._id, COUNT(DISTINCT r._id) AS recent_receipt_count
#     FROM brands b
#     JOIN receipt_items ri ON b.brandCode = ri.brandCode  -- Using brandCode as the linking attribute
#     JOIN Receipts r ON ri.receipt_id = r._id
#     WHERE r.dateScanned >= date('now', 'start of month', '-1 month') 
#           AND r.dateScanned < date('now', 'start of month')
#     GROUP BY b._id
#     ORDER BY recent_receipt_count DESC
#     LIMIT 5
# ), PreviousMonth AS (
#     SELECT b._id, COUNT(DISTINCT r._id) AS previous_receipt_count
#     FROM brands b
#     JOIN receipt_items ri ON b.brandCode = ri.brandCode  -- Using brandCode as the linking attribute
#     JOIN receipts r ON ri.receipt_id = r._id
#     WHERE r.dateScanned >= date('now', 'start of month', '-2 month') 
#           AND r.dateScanned < date('now', 'start of month', '-1 month')
#     GROUP BY b._id
#     ORDER BY previous_receipt_count DESC
#     LIMIT 5
# )
# SELECT rm._id, rm.recent_receipt_count, pm.previous_receipt_count
# FROM RecentMonth rm
# LEFT JOIN PreviousMonth pm ON rm._id = pm._id;
# """
# cursor.execute(query)
# rows = cursor.fetchall()
# for row in rows:
#     print(row)

### Which brand has the most spend among users who were created within the past 6 months?

In [15]:
# cursor = conn.cursor()
# query = """
# SELECT b._id, b.name, SUM(r.totalSpent) AS total_spend
# FROM brands b
# JOIN receipt_items ri ON b.brandCode = ri.brandCode  -- Using brandCode as the linking attribute
# JOIN receipts r ON ri.receipt_id = r._id
# JOIN users u ON r.userId = u._id  -- Correct table name should be 'Users', not 'user'
# WHERE u.createdDate > date('now', '-6 months')
# GROUP BY b._id
# ORDER BY total_spend DESC
# LIMIT 1;
# """
# cursor.execute(query)
# rows = cursor.fetchall()
# for row in rows:
#     print(row)

### Which brand has the most transactions among users who were created within the past 6 months?

In [16]:
# cursor = conn.cursor()
# query = """
# SELECT b._id, b.name, COUNT(DISTINCT r._id) AS transaction_count
# FROM brands b
# JOIN receipt_items ri ON b.brandCode = ri.brandCode  -- Assuming brandCode links Receipt_Items to Brands
# JOIN receipts r ON ri.receipt_id = r._id
# JOIN users u ON r.userId = u._id  -- Ensure the table name is correct, typically 'Users', not 'user'
# WHERE u.createdDate > date('now', '-6 months')
# GROUP BY b._id
# ORDER BY transaction_count DESC
# LIMIT 1;
# """
# cursor.execute(query)
# rows = cursor.fetchall()
# for row in rows:
#     print(row)